# Pipeline ETL: Camada Prata para Ouro

## 1. Objetivo

Este notebook é responsável pela construção do Data Warehouse (Camada Gold). O processo consiste em:
1.  **Extrair** os dados denormalizados da tabela `orders` (Camada Silver).
2.  **Transformar** os dados aplicando a modelagem dimensional (Star Schema), separando os atributos em tabelas Dimensão e as métricas em uma tabela Fato.
3.  **Carregar** os dados no schema `dw` do PostgreSQL, gerando chaves substitutas (Surrogate Keys) para garantir a integridade referencial e performance de BI.

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
import os
from dotenv import load_dotenv
import psycopg2
import warnings

warnings.filterwarnings('ignore')

spark = SparkSession.builder \
    .appName("ETLSilverToGoldOlist") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.5.0") \
    .config("spark.sql.debug.maxToStringFields", 1000) \
    .config("spark.ui.showConsoleProgress", "false") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
print("SparkSession iniciada.")

## 2. Preparação do Ambiente (Schema DW)

Antes de iniciar a carga, executamos o script DDL para garantir que o schema `dw` e as tabelas de destino existam e estejam limpas.

In [2]:
def carregar_env():
    env_path = '../../.env'
    load_dotenv(dotenv_path=env_path)
    return {
        "user": os.getenv("DB_USER"),
        "password": os.getenv("DB_PASSWORD"),
        "host": os.getenv("DB_HOST"),
        "port": os.getenv("DB_PORT"),
        "dbname": os.getenv("DB_NAME")
    }

def resetar_dw(env_vars):
    ddl_path = "../../DataLayer/gold/ddl.sql"
    print("Executando DDL...")
    try:
        conn = psycopg2.connect(**env_vars)
        conn.autocommit = True
        cur = conn.cursor()
        with open(ddl_path, 'r') as f:
            cur.execute(f.read())
        print("Schema DW recriado com sucesso.")
    except Exception as e:
        print(f"Erro no DDL: {e}")
        raise
    finally:
        if 'cur' in locals(): cur.close()
        if 'conn' in locals(): conn.close()

env_vars = carregar_env()
resetar_dw(env_vars)

Executando DDL...
Schema DW recriado com sucesso.


## 3. Extração (Leitura da Silver)

Lemos os dados da tabela `public.orders` para um DataFrame Spark. Esta é a nossa fonte única da verdade.

In [3]:
jdbc_url = f"jdbc:postgresql://{env_vars['host']}:{env_vars['port']}/{env_vars['dbname']}"
jdbc_props = {"user": env_vars['user'], "password": env_vars['password'], "driver": "org.postgresql.Driver"}

print("Lendo dados da camada Silver...")
df_silver = spark.read.jdbc(jdbc_url, "public.orders", properties=jdbc_props)
df_silver.cache()
print(f"Linhas carregadas: {df_silver.count()}")

Lendo dados da camada Silver...
Linhas carregadas: 88981


## 4. Processamento das Dimensões (Transform & Load)

Nesta etapa, "fatiamos" os dados da Silver para criar as tabelas de dimensão. Para cada dimensão:
1.  Selecionamos colunas distintas.
2.  Geramos uma Surrogate Key sequencial.
3.  Carregamos na tabela correspondente no PostgreSQL.

In [4]:
print("Processando e Carregando Dimensões Cliente, Vendedor, Produto, Pagamento...")

dim_cli = df_silver.select(
    F.col("customer_unique_id").alias("ntk_idn_cli"),
    F.col("customer_city").alias("nom_cid"),
    F.col("customer_state").alias("sig_est")
).distinct()
dim_cli.write.jdbc(jdbc_url, "dw.dim_cli", "append", jdbc_props)

dim_vnd = df_silver.select(
    F.col("seller_id").alias("ntk_idn_vnd"),
    F.col("seller_city").alias("nom_cid"),
    F.col("seller_state").alias("sig_est")
).distinct()
dim_vnd.write.jdbc(jdbc_url, "dw.dim_vnd", "append", jdbc_props)

dim_pro = df_silver.select(
    F.col("product_id").alias("ntk_idn_pro"),
    F.col("product_category_name").alias("nom_cat")
).distinct()
dim_pro.write.jdbc(jdbc_url, "dw.dim_pro", "append", jdbc_props)

dim_pag = df_silver.select(
    F.col("payment_type").alias("dsc_tip_pag"),
    F.col("payment_installments").alias("num_par")
).distinct()
dim_pag.write.jdbc(jdbc_url, "dw.dim_pag", "append", jdbc_props)

print("Carga de Dimensões concluída.")

Processando e Carregando Dimensões Cliente, Vendedor, Produto, Pagamento...
Carga de Dimensões concluída.


### 4.1. Dimensão Tempo

A Dimensão Tempo é gerada a partir das datas de compra existentes. Enriquecemos os dados extraindo atributos como Dia, Mês, Ano, Trimestre, Semestre e Flag de Fim de Semana para facilitar a análise temporal no dashboard.

In [5]:
print("Processando e Carregando Dimensão Tempo...")

df_datas = df_silver.select(F.to_date("order_purchase_timestamp").alias("dat_cmp")).distinct()

dim_tmp = df_datas.filter(F.col("dat_cmp").isNotNull()).select(
    F.col("dat_cmp"),
    F.date_format("dat_cmp", "yyyyMMdd").cast("int").alias("srk_tmp"),
    F.year("dat_cmp").alias("num_ano"),
    F.month("dat_cmp").alias("num_mes"),
    F.date_format("dat_cmp", "MMMM").alias("nom_mes"),
    F.dayofmonth("dat_cmp").alias("num_dia"),
    F.quarter("dat_cmp").alias("num_tri"),
    F.date_format("dat_cmp", "EEEE").alias("nom_dia_sem"),
    (F.dayofweek("dat_cmp").isin([1, 7])).alias("flg_fds")
)

dim_tmp.write.jdbc(jdbc_url, "dw.dim_tmp", "append", jdbc_props)
print("Dimensão Tempo carregada.")

Processando e Carregando Dimensão Tempo...
Dimensão Tempo carregada.


## 5. Processamento da Tabela Fato (Transform & Load)

Esta é a etapa final de integração. Realizamos o **JOIN** da tabela Silver original com as novas Dimensões criadas para substituir as chaves naturais (IDs originais) pelas chaves artificiais do DW.

Selecionamos apenas as chaves estrangeiras e as métricas numéricas para compor a tabela `dw.fat_ped`.

In [6]:
print("Processando Tabela Fato...")
print("Lendo dimensões do DW (com SKs)...")

dim_cli_sk = spark.read.jdbc(jdbc_url, "dw.dim_cli", properties=jdbc_props)
dim_vnd_sk = spark.read.jdbc(jdbc_url, "dw.dim_vnd", properties=jdbc_props)
dim_pro_sk = spark.read.jdbc(jdbc_url, "dw.dim_pro", properties=jdbc_props)
dim_pag_sk = spark.read.jdbc(jdbc_url, "dw.dim_pag", properties=jdbc_props)
dim_tmp_sk = spark.read.jdbc(jdbc_url, "dw.dim_tmp", properties=jdbc_props)

df_fato_base = df_silver.withColumn("data_join", F.to_date("order_purchase_timestamp"))

print("Fazendo JOINs da Fato com as Dimensões...")
df_fato = df_fato_base \
    .join(dim_cli_sk, df_fato_base.customer_unique_id == dim_cli_sk.ntk_idn_cli, "left") \
    .join(dim_vnd_sk, df_fato_base.seller_id == dim_vnd_sk.ntk_idn_vnd, "left") \
    .join(dim_pro_sk, df_fato_base.product_id == dim_pro_sk.ntk_idn_pro, "left") \
    .join(dim_tmp_sk, df_fato_base.data_join == dim_tmp_sk.dat_cmp, "left") \
    .join(dim_pag_sk, 
          (df_fato_base.payment_type == dim_pag_sk.dsc_tip_pag) & 
          (df_fato_base.payment_installments == dim_pag_sk.num_par), "left")

print("Selecionando colunas finais da Fato...")
fat_ped = df_fato.select(
    F.col("srk_cli"),
    F.col("srk_vnd"),
    F.col("srk_pro"),
    F.col("srk_tmp"),
    F.col("srk_pag"),
    F.col("order_id").alias("ntk_idn_ped"),
    (F.col("price") + F.col("freight_value")).alias("vlr_tot"),
    F.col("freight_value").alias("vlr_frt"),
    F.col("price").alias("vlr_itm"),
    F.col("delivery_days").alias("qtd_dia_ent"),
    F.col("review_score").alias("num_ava"),
    F.col("is_delivery_late").alias("flg_atr")
)

print("Carregando Tabela Fato...")
fat_ped.write.jdbc(jdbc_url, "dw.fat_ped", "append", jdbc_props)
print("Carga da Camada Gold concluída com sucesso.")

Processando Tabela Fato...
Lendo dimensões do DW (com SKs)...
Fazendo JOINs da Fato com as Dimensões...
Selecionando colunas finais da Fato...
Carregando Tabela Fato...
Carga da Camada Gold concluída com sucesso.


In [7]:
spark.stop()